In [1]:
pip install tweeterpy

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import csv
from tweeterpy import TweeterPy
from datetime import datetime, date, timezone
import unicodedata

twitter = TweeterPy()
database = twitter.search(search_query='lacoste funk', #Edita aqui com a palavra ou frase que você quiser
                          end_cursor=None,
                          total=10, #Aqui é para limitar o número máximo de resultados que você quer buscar. Se não tiver um máximo, você coloca None
                          search_filter='Latest') # Latest, Top, People, Photos, Videos (Product), tem que escrever com a maiúscula

# Definir os limites da data de publicação desejada
start_date = datetime(2020, 6, 1, tzinfo=timezone.utc) #Aqui você muda para filtrar data de início, formato ANO, MêS, DIA
end_date = datetime(2023, 6, 30, tzinfo=timezone.utc) #Aqui você muda para filtrar data de fim, formato ANO, MêS, DIA

# Extrair os dados relevantes do dicionário
data = []
for entry in database['data']:
    tweet = entry['content']['itemContent']['tweet_results']['result']
    tweet_id = tweet['rest_id']
    created_at = tweet['legacy']['created_at']
    created_datetime = datetime.strptime(created_at, "%a %b %d %H:%M:%S %z %Y").replace(tzinfo=timezone.utc)
    if start_date <= created_datetime <= end_date:
        user = tweet['core']['user_results']['result']
        user_name = user['legacy']['name']
        full_text = f"{user_name}: {tweet['legacy']['full_text']}"
        full_text = full_text.replace('\n', ' ')
        full_text = unicodedata.normalize('NFKD', full_text).encode('ascii', 'ignore').decode('utf-8')
        data.append([tweet_id, created_at, full_text])

# Criar o DataFrame a partir dos dados
df = pd.DataFrame(data, columns=['Tweet ID', 'Created At', 'Full Text'])

# Remover linhas em branco
df.dropna(subset=['Tweet ID', 'Created At', 'Full Text'], inplace=True)

# Exportar para um arquivo CSV
df.to_csv('tweets.csv', index=False, sep="|", quoting=csv.QUOTE_ALL, encoding='utf-8')


API Updated Successfully.
